# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import gmaps 
import requests 

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
#Opening the csv file 
csv_path = "city_data.csv" 
city_data_df = pd.read_csv(csv_path)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
#Setting cordinates equal to the Latitude and Longitude 
cordinates = city_data_df[['Latitude','Longitude']]

In [9]:
#Setting the map parameters for the google maps 
figure_layout = {
    'width' : '400px',
    'height' : '300px',
    'border' : '1px solid black',
    'padding' :'1px',
    'margin' : '0 auto 0 auto'
}

#Making the google maps 
fig = gmaps.figure(layout = figure_layout)

In [11]:
#Setting fig equal to gmaps creation 
fig = gmaps.figure()

#Setting humidity equal to Humidity 
humidity = city_data_df['Humidity']

#Making a heat map layer
heat_layer = gmaps.heatmap_layer(cordinates, weights = humidity, 
                                 dissipating = False, max_intensity = 91, 
                                 point_radius = 1)

#Adding the heat map layer to your map 
fig.add_layer(heat_layer)

#Displaying the figure 
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
#Filtering all the data of cities that have a temperature between 80°F and 70°F
city_data_df = city_data_df.loc[city_data_df['Temperature °F']>70]
city_data_df = city_data_df.loc[city_data_df['Temperature °F']<80]

#Filtering all the data of cities that have wind speed less than 10mph
city_data_df = city_data_df.loc[city_data_df['Wind Speed']<10]

#Filtering all the data of cities that have no cloudiness 
city_data_df = city_data_df.loc[city_data_df['Cloudiness']==0]

In [13]:
#Dropping rows with NA
hotel_df = city_data_df.dropna()

#Displaying hotel_df
hotel_df

,Unnamed: 0,City,Temperature °F,Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Country,Date
19,19,Gerāsh,78.80,50,0,1.30,27.67,54.14,IR,1601354144
111,111,San Quintín,72.43,70,0,9.04,30.48,-115.95,MX,1601353893
263,263,Erzin,73.99,91,0,1.90,36.96,36.20,TR,1601354140
268,268,Carutapera,77.86,81,0,7.90,-1.20,-46.02,BR,1601354444
326,326,Morondava,77.47,52,0,4.34,-20.28,44.28,MG,1601353992
441,441,Birao,71.22,93,0,4.27,10.28,22.79,CF,1601354501
529,529,Abu Samrah,77.00,46,0,5.03,35.30,37.18,SY,1601354530
545,545,Quelimane,73.40,88,0,2.24,-17.88,36.89,MZ,1601354535
564,564,Al Bardīyah,79.65,52,0,6.38,31.76,25.09,LY,1601354540


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
#Putting the hotel cordinates into a zip 
hotel_cordinates = zip(hotel_df['Latitude'],hotel_df['Longitude']) 

#Setting the parameters for google maps 
target_radius = 5000
target_type = 'lodging'

#Making hotel_name a list 
hotel_name = []

#Looping through all the cordinates to find the closest hotel within 5000 meters 
for hotel_cordinates in hotel_cordinates:
    
    
    try:
        #Converting the parameters into a dictionary 
        params = {
            'location' : (f'{hotel_cordinates[0]}, {hotel_cordinates[1]}'),
            'radius' : target_radius,
            'type' : target_type,
            'key' : g_key
        }
        
        #Setting the base url to the url 
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
        #Requesting and finding the name of each hotel 
        response = requests.get(base_url, params=params)
        hotel_data = response.json()
        hotel_name.append(hotel_data['results'][0]['name'])
    
    #Using an except incase there is no hotel within 5000 meters 
    except IndexError:
        
        #Print the cordinates of the areas where hotels were not found within 5000 meters
        print(f'{hotel_cordinates[0]}, {hotel_cordinates[1]} Does not exist')

#Displaying the list of hotel names        
hotel_name

10.28, 22.79 Does not exist
35.3, 37.18 Does not exist


['Chelpasin',
 'Old Mill Hotel',
 'Artemis Otel',
 'Lidera',
 'La Case Bambou',
 'Hotel Flamingo',
 'بورتو بردية']

In [15]:
#Dropping cities where a hotel was not found 
hotel_df = hotel_df.drop([441, 529])

#Adding Hotel Name to the end of the exisiting Data Frame 
hotel_df['Hotel Name'] = hotel_name

#Dropping the Unamed Field 
hotel_df = hotel_df.drop(['Unnamed: 0'], axis=1)

#Displaying the data frame 
hotel_df

,City,Temperature °F,Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Country,Date,Hotel Name
19,Gerāsh,78.80,50,0,1.30,27.67,54.14,IR,1601354144,Chelpasin
111,San Quintín,72.43,70,0,9.04,30.48,-115.95,MX,1601353893,Old Mill Hotel
263,Erzin,73.99,91,0,1.90,36.96,36.20,TR,1601354140,Artemis Otel
268,Carutapera,77.86,81,0,7.90,-1.20,-46.02,BR,1601354444,Lidera
326,Morondava,77.47,52,0,4.34,-20.28,44.28,MG,1601353992,La Case Bambou
545,Quelimane,73.40,88,0,2.24,-17.88,36.89,MZ,1601354535,Hotel Flamingo
564,Al Bardīyah,79.65,52,0,6.38,31.76,25.09,LY,1601354540,بورتو بردية


In [16]:
#Setting Locations equal to the Latitide and Longitude 
locations = hotel_df[['Latitude','Longitude']]

#Setting fig equal to gmaps creation 
fig = gmaps.figure()

#Setting humidity equal to Humidity 
humidity = hotel_df['Humidity']

#Making a heat map layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating = False, max_intensity = 91, 
                                 point_radius = 1)

#Adding the heat map layer to your map 
fig.add_layer(heat_layer)

#Displaying the figure 
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[['Latitude','Longitude']]

In [18]:
#Making the marker layer
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add marker layer ontop of heat map
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))